# Modeling - Predicting Taxi Trip Durations in NYC

Description...

Outline...

- ...

TODO: maybe there's a way to get features via a maps API

## Set up

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()
sns.set_style('whitegrid')

In [2]:
from load_preprocess_data import load_train_data, load_test_data

# load data
train_data = load_train_data('data/W22P1_train.csv')
test_data = load_test_data('data/W22P1_test.csv')

In [3]:
train_data.head()

,pickup_datetime,dayofweek,hour,passenger_count,distance_km,l1_distance_km,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,log_trip_duration
id,,,,,,,,,,,,
0,2016-01-07 19:32:15,3,19,1,1.259700,1.997172,-73.986389,40.756615,-73.999794,40.761631,520,6.253829
1,2016-01-27 08:07:32,2,8,1,2.356650,2.513306,-73.956039,40.767609,-73.968201,40.786690,989,6.896694
2,2016-01-31 13:52:55,6,13,1,2.806862,2.918886,-73.975998,40.751137,-74.001854,40.735229,657,6.487684
3,2016-01-19 08:00:19,1,8,3,3.155510,4.015846,-73.960121,40.781952,-73.971970,40.755039,1035,6.942157
4,2016-01-25 23:32:14,0,23,1,1.725446,2.188162,-73.987434,40.760139,-73.990982,40.744862,621,6.431331


In [4]:
all_covariates = list(test_data.columns)

original_covariates = ['hour', 'passenger_count',
                       'pickup_longitude', 'pickup_latitude',
                       'dropoff_longitude', 'dropoff_latitude']

# numerical covariates
numerical_covariates = ['hour', 'passenger_count',
                        'distance_km', 'l1_distance_km',
                        'pickup_longitude', 'pickup_latitude',
                        'dropoff_longitude', 'dropoff_latitude']

# categorical + numerical covariates
cat_numerical_covariates = ['dayofweek', 'hour',
                            'passenger_count', 'distance_km', 'l1_distance_km',
                            'pickup_longitude', 'pickup_latitude',
                            'dropoff_longitude', 'dropoff_latitude']

print('covariates: ', all_covariates)

covariates:  ['pickup_datetime', 'dayofweek', 'hour', 'passenger_count', 'distance_km', 'l1_distance_km', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']


In [5]:
# train-test split the training data (so that we can evaluate without submitting)
from sklearn.model_selection import train_test_split
train_train_data, train_test_data = train_test_split(train_data, test_size=0.1)

In [6]:
def create_X_y(train_data, test_data, covariates, label):
    X_train = train_data[covariates]
    X_test = test_data[covariates]

    y_train = train_data[label]
    y_test = test_data[label]

    return X_train, X_test, y_train, y_test

In [7]:
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_squared_error

def eval_model(model, X, y, metric='rmsle', log=False):
    '''evaluate model on given model via the given metric'''

    y_pred = model.predict(X)
    if log:
        y_pred = np.exp(y_pred)
        y = np.exp(y)

    if metric=='rmsle':
        return np.sqrt(mean_squared_log_error(y, y_pred))
    elif 'msle':
        return mean_squared_log_error(y, y_pred)
    elif metric=='mse':
        return mean_squared_error(y, y_pred)
    elif metric=='rmse':
        return np.sqrt(mean_squared_error(y, y_pred))
    elif metric=='mae':
        return mean_absolute_error(y, y_pred)
    else:
        raise ValueError()

In [8]:
def create_submission(model, covariates, log=False):
    X_test = test_data[covariates]
    y_pred = model.predict(X_test)

    if log:
        y_pred = np.exp(y_pred)

    df = pd.DataFrame(index=test_data.index, data=y_pred, columns=['trip_duration'])

    return df

## Baseline Models

### Linear Regression on Original Features

In [9]:
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = create_X_y(train_train_data, train_test_data, original_covariates, 'log_trip_duration')

reg = LinearRegression().fit(X_train, y_train)

print('train rmsle: ', eval_model(reg, X_train, y_train, metric='rmsle', log=True))
print('test rmsle: ', eval_model(reg, X_test, y_test, metric='rmsle', log=True))

train rmsle:  0.7451294021653776
test rmsle:  0.7191473461365062


## Linear Regression - Haversine Distance Only

In [10]:
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = create_X_y(train_train_data, train_test_data, ['distance_km'], 'log_trip_duration')

reg = LinearRegression().fit(X_train, y_train)

print('train rmsle: ', eval_model(reg, X_train, y_train, metric='rmsle', log=True))
print('test rmsle: ', eval_model(reg, X_test, y_test, metric='rmsle', log=True))

train rmsle:  0.6256579674755794
test rmsle:  0.602153602733774


## Linear Regression - All Features

In [11]:
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = create_X_y(train_train_data, train_test_data, numerical_covariates, 'log_trip_duration')

reg = LinearRegression().fit(X_train, y_train)

print('train rmsle: ', eval_model(reg, X_train, y_train, metric='rmsle', log=True))
print('test rmsle: ', eval_model(reg, X_test, y_test, metric='rmsle', log=True))

train rmsle:  0.6128507543721159
test rmsle:  0.5853723012574025


## Linear Regression - ElasticNet

In [12]:
from sklearn.linear_model import ElasticNetCV

X_train, X_test, y_train, y_test = create_X_y(train_train_data, train_test_data, numerical_covariates, 'log_trip_duration')

reg = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 1], n_alphas=100, cv=10).fit(X_train, y_train)

print('train rmsle: ', eval_model(reg, X_train, y_train, metric='rmsle', log=True))
print('test rmsle: ', eval_model(reg, X_test, y_test, metric='rmsle', log=True))

train rmsle:  0.619604372211765
test rmsle:  0.5902675055847393


In [13]:
np.array(numerical_covariates)[reg.coef_ != 0] # selected features

array(['hour', 'passenger_count', 'distance_km', 'l1_distance_km'],
      dtype='<U17')

## Linear Regression - Recursive Feature Elimination

In [14]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = create_X_y(train_train_data, train_test_data, numerical_covariates, 'log_trip_duration')

reg = RFECV(LinearRegression(), min_features_to_select=1, cv=10).fit(X_train, y_train)

print('train rmsle: ', eval_model(reg, X_train, y_train, metric='rmsle', log=True))
print('test rmsle: ', eval_model(reg, X_test, y_test, metric='rmsle', log=True))

train rmsle:  0.6128507543721159
test rmsle:  0.5853723012574025


In [15]:
reg.support_ # selected features

array([ True,  True,  True,  True,  True,  True,  True,  True])

## Random Forest - All Features

Training random forests on sklearn is slow, let's just try one

In [16]:
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = create_X_y(train_train_data, train_test_data, cat_numerical_covariates, 'log_trip_duration')

reg = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)

print('train rmsle: ', eval_model(reg, X_train, y_train, metric='rmsle', log=True))
print('test rmsle: ', eval_model(reg, X_test, y_test, metric='rmsle', log=True))

train rmsle:  0.17794091712065188
test rmsle:  0.446359104968922


Feature Importances

In [17]:
pd.DataFrame(index=reg.feature_names_in_, data=reg.feature_importances_,
                columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

,feature_importance
distance_km,0.604150
l1_distance_km,0.080790
dropoff_latitude,0.065780
hour,0.056010
dropoff_longitude,0.051882
pickup_latitude,0.050449
pickup_longitude,0.049143
dayofweek,0.029342
passenger_count,0.012452


This validates that the 'distance_km' feature is highly predictive.

## Gradient Boosted Machines (use XGBoost)

In [18]:
import xgboost as xgb

X_train, X_test, y_train, y_test = create_X_y(train_train_data, train_test_data, cat_numerical_covariates, 'log_trip_duration')

reg = xgb.XGBRegressor(objective='reg:squarederror', importance_type='total_gain', n_jobs=-1).fit(X_train, y_train)

print('train rmsle: ', eval_model(reg, X_train, y_train, metric='rmsle', log=True))
print('test rmsle: ', eval_model(reg, X_test, y_test, metric='rmsle', log=True))

C:\Users\awnya\miniconda3\envs\stat457\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\awnya\miniconda3\envs\stat457\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


train rmsle:  0.32099335873310375
test rmsle:  0.4482136967534261


C:\Users\awnya\miniconda3\envs\stat457\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [19]:
pd.DataFrame(index=X_train.columns, data=reg.feature_importances_,
                columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

,feature_importance
distance_km,0.704744
l1_distance_km,0.078083
hour,0.042820
dropoff_latitude,0.041156
pickup_longitude,0.038024
dropoff_longitude,0.032301
pickup_latitude,0.032104
dayofweek,0.025648
passenger_count,0.005120


## XGBoost GridSearch (with Original Features)

In [20]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = create_X_y(train_train_data, train_test_data, original_covariates, 'log_trip_duration')

params = {'n_estimators': [20, 30, 40, 50, 60], 'max_depth': [4, 5, 6, 7]}
reg = xgb.XGBRegressor(objective='reg:squarederror', importance_type='total_gain')

cv = GridSearchCV(reg, params, cv=5, n_jobs=-1).fit(X_train, y_train)

print('train rmsle: ', eval_model(cv, X_train, y_train, metric='rmsle', log=True))
print('test rmsle: ', eval_model(cv, X_test, y_test, metric='rmsle', log=True))

C:\Users\awnya\miniconda3\envs\stat457\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


train rmsle:  0.4316321449449939
test rmsle:  0.4799301530467915


C:\Users\awnya\miniconda3\envs\stat457\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Not quite as good as with the added features (incl `distance_km`), but not that much worse.

Feature importance without the added features:

In [21]:
pd.DataFrame(index=X_train.columns, data=cv.best_estimator_.feature_importances_,
                columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

,feature_importance
pickup_latitude,0.274408
pickup_longitude,0.265895
dropoff_longitude,0.241579
dropoff_latitude,0.175665
hour,0.039182
passenger_count,0.003270


Location-related features were most important. It's as if the model is recreating the `distance_km` feature. Those features become much less important when `distance_km` is included.

## XGBoost GridSearch (all features)

In [22]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = create_X_y(train_train_data, train_test_data, cat_numerical_covariates, 'log_trip_duration')

params = {
    'n_estimators': [60, 80],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.5, 1.0, 1.5],
    # 'subsample': [0.9],
    # 'colsample_bytree': [0.8, 0.9, 1],
    # 'gamma': [0, 1, 5]
}

reg = xgb.XGBRegressor(objective='reg:squarederror', importance_type='total_gain')

cv = GridSearchCV(reg, params, cv=5, n_jobs=-1, verbose=1).fit(X_train, y_train)

print('train rmsle: ', eval_model(cv, X_train, y_train, metric='rmsle', log=True))
print('test rmsle: ', eval_model(cv, X_test, y_test, metric='rmsle', log=True))

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\awnya\miniconda3\envs\stat457\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


train rmsle:  0.43467274066404016
test rmsle:  0.44486305260802245


C:\Users\awnya\miniconda3\envs\stat457\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [23]:
pd.DataFrame(index=X_train.columns, data=cv.best_estimator_.feature_importances_,
                columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

,feature_importance
distance_km,0.780927
l1_distance_km,0.097191
hour,0.039640
dropoff_latitude,0.023518
pickup_longitude,0.018028
dayofweek,0.016689
dropoff_longitude,0.011645
pickup_latitude,0.011578
passenger_count,0.000785


## XGBoost Recursive Feature Elimination

In [24]:
from sklearn.feature_selection import RFECV
import xgboost as xgb

X_train, X_test, y_train, y_test = create_X_y(train_train_data, train_test_data, numerical_covariates, 'log_trip_duration')

reg = xgb.XGBRegressor(objective='reg:squarederror', importance_type='total_gain')
rfecv = RFECV(reg, min_features_to_select=1, cv=10).fit(X_train, y_train)

print('train rmsle: ', eval_model(rfecv, X_train, y_train, metric='rmsle', log=True))
print('test rmsle: ', eval_model(rfecv, X_test, y_test, metric='rmsle', log=True))

train rmsle:  0.34244487607368546
test rmsle:  0.460820634011596


In [25]:
rfecv.support_

array([ True,  True,  True,  True,  True,  True,  True,  True])